In [1]:
from ogb.nodeproppred import PygNodePropPredDataset
import os
import os.path as osp
import torch_geometric
import torch
import pandas as pd
import numpy as np
from torch_geometric.data import InMemoryDataset, download_url
import random

from encoder import SentenceEncoder

In [2]:
# Loading the ogbn-arxiv dataset
data_root = "../data"
arxiv = PygNodePropPredDataset(name='ogbn-arxiv', root=data_root)

In [3]:
dataset = arxiv[0]
dataset

Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343, 1])

In [4]:
def get_taxonomy(data_root):
    # read categories and description file
    f = open(osp.join(data_root, "ogbn_arxiv", "arxiv_CS_categories.txt"), "r").readlines()

    state = 0
    result = {"id": [], "name": [], "description": []}

    for line in f:
        if state == 0:
            assert line.strip().startswith("cs.")
            category = ("arxiv "
                + " ".join(line.strip().split(" ")[0].split(".")).lower())
            # e.g. cs lo

            name = line.strip()[7:-1]  # e. g. Logic in CS
            result["id"].append(category)
            result["name"].append(name)
            state = 1
            continue

        elif state == 1:
            description = line.strip()
            result["description"].append(description)
            state = 2
            continue

        elif state == 2:
            state = 0
            continue

    arxiv_cs_taxonomy = pd.DataFrame(result)
    return arxiv_cs_taxonomy

In [5]:
def get_node_feature(data_root):
    nodeidx2paperid = pd.read_csv(osp.join(data_root, "ogbn_arxiv/mapping/nodeidx2paperid.csv.gz"), index_col="node idx")

    # Load the title and abstract of each paper
    titleabs_url = "https://snap.stanford.edu/ogb/data/misc/ogbn_arxiv/titleabs.tsv"
    titleabs_path = osp.join(data_root, "ogbn_arxiv", "titleabs.tsv")

    if (osp.exists(titleabs_path)):
        titleabs = pd.read_csv(titleabs_path, sep="\t", names=["paper id", "title", "abstract"], index_col="paper id")
    else:
        titleabs = pd.read_csv(titleabs_url, sep="\t", names=["paper id", "title", "abstract"], index_col="paper id")

    titleabs = nodeidx2paperid.join(titleabs, on="paper id")


    # Prompt for the feature of nodes (can be changed accordingly)
    node_feature_prompt = ("Feature Node.\n"
                        + "Paper Title and Abstract : "
                        + titleabs["title"]
                        + " + "
                        + titleabs["abstract"])

    node_feature_prompt_list = node_feature_prompt.values
    return node_feature_prompt_list

In [6]:
def get_label_feature(data_root):
    arxiv_cs_taxonomy = get_taxonomy(data_root)

    mapping_file = osp.join(data_root, "ogbn_arxiv", "mapping", "labelidx2arxivcategeory.csv.gz")
    labelidx2arxivcategory = pd.read_csv(mapping_file)

    arxiv_categ_vals = pd.merge(labelidx2arxivcategory, arxiv_cs_taxonomy, left_on="arxiv category", right_on="id")


    # Prompt for the label nodes (can be changed accordingly)
    label_node_prompt = ("Prompt Node.\n"
                        + "Literature Category and Description: "
                        + arxiv_categ_vals["name"]
                        + " + "
                        + arxiv_categ_vals["description"])

    label_node_prompt_list = label_node_prompt.values
    return label_node_prompt_list

In [8]:
class ArxivPyGDataset(InMemoryDataset):
    def __init__(self, dataRoot="../data", custom_dataRoot="../custom_data", sentence_encoder=None, transform=None, pre_transform=None, pre_filter=None):
        self.data_root = dataRoot
        self.custom_data_root = custom_dataRoot
        self.sentence_encoder = sentence_encoder
        self.custom_data_dir = osp.join(self.custom_data_root, f"ogbn_arxiv_{self.sentence_encoder.name}")

        if not osp.exists(self.custom_data_dir):
            os.makedirs(self.custom_data_dir)

        super().__init__(self.custom_data_dir, transform, pre_transform, pre_filter)

        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ["data.pt", "texts.pkl"]

    def text_to_embed(self, texts):
        if self.sentence_encoder is None:
            raise NotImplementedError("Sentence Encoder is not passed")
        if texts is None:
            return None
        else:
            return self.sentence_encoder.encode(texts) # returns to self.device

    def encode_texts(self, texts):
        if isinstance(texts[0], str):
            return self.text_to_embed(texts)
        return [self.text_to_embed(t) for t in texts]

    def generate_custom_data(self):
        node_texts = get_node_feature(self.data_root).tolist()
        label_texts = get_label_feature(self.data_root).tolist()

        # Prompt for prompt node/edge and edge texts (can be changed accordingly)
        edge_texts = ["Feature Edge.\n Citation"]
        prompt_texts = ["Prompt Node.\n Node Classification of Literature Category"]
        prompt_edge_texts = ["Prompt Edge."]

        return [node_texts, label_texts, edge_texts, prompt_texts, prompt_edge_texts]

    def process(self):
        arxiv_data = PygNodePropPredDataset(name="ogbn-arxiv", root=self.data_root)
        arxiv_data_list = arxiv_data._data
        arxiv_data_list.y = arxiv_data_list.y.squeeze()  # to flatten the y tensor

        texts = self.generate_custom_data()
        texts_embed = self.encode_texts(texts)

        torch.save(texts, self.processed_paths[1])

        arxiv_data_list.x_text_feat = texts_embed[0] # node text feature
        arxiv_data_list.label_text_feat = texts_embed[1] # label text feature
        arxiv_data_list.edge_text_feat = texts_embed[2] # edge text feature
        arxiv_data_list.prompt_text_feat = texts_embed[3] # prompt node text feature
        arxiv_data_list.prompt_edge_feat = texts_embed[4] # prompt edge text feature

        # get dataset split
        split_idx = arxiv_data.get_idx_split()
        train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
        # Generate 10 different permutations of train/valid/test split
        train_idx_list = [train_idx[torch.randperm(train_idx.shape[0])] for _ in range(10)]
        valid_idx_list = [valid_idx[torch.randperm(valid_idx.shape[0])] for _ in range(10)]
        test_idx_list = [test_idx[torch.randperm(test_idx.shape[0])] for _ in range(10)]

        arxiv_data_list.train_masks = train_idx_list
        arxiv_data_list.val_masks = valid_idx_list
        arxiv_data_list.test_masks = test_idx_list

        data, slices = self.collate([arxiv_data_list]) # Pass the data_list as a list

        torch.save((data, slices), self.processed_paths[0])
        print("Arxiv is processed. Saved.")

In [9]:
LMencoder = SentenceEncoder(root="../lang_models", name="ST", device=0)
custom_arxiv = ArxivPyGDataset(dataRoot=data_root, sentence_encoder=LMencoder)

In [10]:
LMencoder2 = SentenceEncoder(root="../lang_models", name="roberta", device=0)
custom_arxiv2 = ArxivPyGDataset(dataRoot=data_root, sentence_encoder=LMencoder2)

In [11]:
arxiv = custom_arxiv._data
arxiv

Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343], x_text_feat=[169343, 768], label_text_feat=[40, 768], edge_text_feat=[1, 768], prompt_text_feat=[1, 768], prompt_edge_feat=[1, 768], train_masks=[10], val_masks=[10], test_masks=[10])

In [12]:
split_path = osp.join(data_root, "ogbn_arxiv", "split", "time")

train_path = osp.join(split_path, "train.csv.gz")
valid_path = osp.join(split_path, "valid.csv.gz")
test_path = osp.join(split_path, "test.csv.gz")

In [13]:
train_idx = pd.read_csv(train_path, compression="gzip", header=None).values.squeeze()
train_idx

array([     0,      1,      2, ..., 169145, 169148, 169251])

In [14]:
train_idx.shape

(90941,)

In [15]:
train_idx = torch.Tensor(train_idx).type(torch.int)
train_idx

tensor([     0,      1,      2,  ..., 169145, 169148, 169251],
       dtype=torch.int32)

In [16]:
train_idx_list = [train_idx[torch.randperm(train_idx.shape[0])] for _ in range(10)]
train_idx_list

[tensor([ 75725,  96878, 143774,  ...,  56262, 167085,  14710],
        dtype=torch.int32),
 tensor([ 80301,  74643, 135429,  ...,  68080, 157601,  83789],
        dtype=torch.int32),
 tensor([100187,  64022,  84793,  ..., 126675, 144475, 106046],
        dtype=torch.int32),
 tensor([  6742, 107581, 163094,  ..., 161658,  57453, 150625],
        dtype=torch.int32),
 tensor([125837, 125939,  67942,  ...,  18600, 155247, 137965],
        dtype=torch.int32),
 tensor([ 22988,  35298, 157988,  ...,  69873,  40173,  39958],
        dtype=torch.int32),
 tensor([  9247, 164880, 162933,  ..., 118699, 153956, 157571],
        dtype=torch.int32),
 tensor([116904, 128611, 125916,  ...,  20010,   1066,  99394],
        dtype=torch.int32),
 tensor([  6099,  53158,  60685,  ...,  31903, 134991, 130488],
        dtype=torch.int32),
 tensor([ 62644,  33167,  98725,  ...,  96238, 152349,  98719],
        dtype=torch.int32)]

In [17]:
arxiv_gpu = arxiv.to("cuda:2")
arxiv_gpu

Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343], x_text_feat=[169343, 768], label_text_feat=[40, 768], edge_text_feat=[1, 768], prompt_text_feat=[1, 768], prompt_edge_feat=[1, 768], train_masks=[10], val_masks=[10], test_masks=[10])

In [18]:
print(arxiv.x)
print(arxiv_gpu.x)

tensor([[-0.0579, -0.0525, -0.0726,  ...,  0.1734, -0.1728, -0.1401],
        [-0.1245, -0.0707, -0.3252,  ...,  0.0685, -0.3721, -0.3010],
        [-0.0802, -0.0233, -0.1838,  ...,  0.1099,  0.1176, -0.1399],
        ...,
        [-0.2205, -0.0366, -0.4022,  ...,  0.1134, -0.1614, -0.1452],
        [-0.1382,  0.0409, -0.2518,  ..., -0.0893, -0.0413, -0.3761],
        [-0.0299,  0.2684, -0.1611,  ...,  0.1208,  0.0776, -0.0910]],
       device='cuda:2')
tensor([[-0.0579, -0.0525, -0.0726,  ...,  0.1734, -0.1728, -0.1401],
        [-0.1245, -0.0707, -0.3252,  ...,  0.0685, -0.3721, -0.3010],
        [-0.0802, -0.0233, -0.1838,  ...,  0.1099,  0.1176, -0.1399],
        ...,
        [-0.2205, -0.0366, -0.4022,  ...,  0.1134, -0.1614, -0.1452],
        [-0.1382,  0.0409, -0.2518,  ..., -0.0893, -0.0413, -0.3761],
        [-0.0299,  0.2684, -0.1611,  ...,  0.1208,  0.0776, -0.0910]],
       device='cuda:2')


In [19]:
arxiv_gpu.x_text_feat.device

device(type='cuda', index=2)

In [20]:
b = arxiv.y.squeeze()
b

tensor([ 4,  5, 28,  ..., 10,  4,  1], device='cuda:2')

In [21]:
arx = PygNodePropPredDataset(name="ogbn-arxiv", root=data_root)
arx

PygNodePropPredDataset()

In [24]:
arx.get_idx_split()

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

: 